In [1]:
import pandas as pd
import sqlalchemy as sal

engine = sal.create_engine('mssql://DESKTOP-JUPCVM3\SQLEXPRESS/master?driver=ODBC+DRIVER+13+FOR+SQL+SERVER')
conn=engine.connect()

In [51]:
#SCD 2 ETL process

def extract():
    df_product = pd.read_csv('products.txt') #create df from received file
    df_product_db = pd.read_sql_query("select * from product_dim where end_date = '9999-12-31'",conn) #extract only active records from table as only those needs to be updated further
    return df_product,df_product_db

def transform(df_product,df_product_db):
    df_merge = pd.merge(df_product,df_product_db, how = 'inner', left_on ='product_id' , right_on ='product_id')
    keys = df_merge['product_key'].to_list()  #convert series to list #find product_key(surrogate keys) for which end_date needs to be updated
    product_keys = ','.join([str(i) for i in keys])  #convert list to comma seperated values
    return product_keys

def updates(product_keys):  #need to update the end date of existing products to current_date-1
    query = sal.text("update product_dim set end_date= cast(getdate()-1 as date), active_flag = 0 where product_key in (" + product_keys +")") #update the end_date of active records before inserting the new data
    conn.execute(query)  
    conn.commit()

def insert(df_product):   #need to insert all the records with start_date = current_date & end_date = 9999-12-31
    df_product['start_date'] = pd.to_datetime('now').strftime('%Y-%m-%d')  #add start_date as current_date
    df_product['end_date'] = '9999-12-31'   #add end_date as future date
    df_product['active_flag'] = 1
    df_product.to_sql('product_dim',conn,index = False,if_exists = 'append') #insert new data with start_date and end_date
    conn.commit()


In [52]:
df_product,df_product_db = extract()

product_keys = transform(df_product,df_product_db)

if product_keys != '':
    updates(product_keys)

insert(df_product)

In [42]:
pd.to_datetime('now').strftime('%Y-%m-%d')

'2023-09-28'

In [37]:
product_keys

'1,2'

In [38]:
"update product_dim set end_date= cast(getdate()-1 as date) where product_key in (" + product_keys +")"

'update product_dim set end_date= cast(getdate()-1 as date) where product_key in (1,2)'

In [33]:
df_merge['product_key']

#convert series to list
keys = df_merge['product_key'].to_list()

#convert list to comma seperated values
product_keys = ','.join([str(i) for i in keys])

# update product_dim set end_date= cast(getdate()-1 as date) where product_key in ()

In [34]:
product_keys

'1,2'

In [26]:
a = ['1','2']
','.join(a)  #this works only with strings but not with int

'1,2'

In [28]:
a = [1,2]
[str(i) for i in a]  #list comprehension #to convert each element to string value

['1', '2']

In [29]:
','.join([str(i) for i in a])

'1,2'